In [1]:
"""
The MIT License (MIT)
Copyright (c) 2021 NVIDIA
Permission is hereby granted, free of charge, to any person obtaining a copy of
this software and associated documentation files (the "Software"), to deal in
the Software without restriction, including without limitation the rights to
use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of
the Software, and to permit persons to whom the Software is furnished to do so,
subject to the following conditions:
The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS
FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR
COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER
IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN
CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
"""


'\nThe MIT License (MIT)\nCopyright (c) 2021 NVIDIA\nPermission is hereby granted, free of charge, to any person obtaining a copy of\nthis software and associated documentation files (the "Software"), to deal in\nthe Software without restriction, including without limitation the rights to\nuse, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of\nthe Software, and to permit persons to whom the Software is furnished to do so,\nsubject to the following conditions:\nThe above copyright notice and this permission notice shall be included in all\ncopies or substantial portions of the Software.\nTHE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR\nIMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS\nFOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR\nCOPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER\nIN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OU

This code example is very similar to c5e1_mnist_learning but the network is modified to use ReLU neruons in the hidden layer, softmax in the output layer, categorical crossentropy as loss function, Adam as optimizer, and a mini-batch size of 64. More context for this code example can be found in the section "Experiment: Tweaking Network and Learning Parameters" in Chapter 5 in the book Learning Deep Learning by Magnus Ekman (ISBN: 9780137470358).

In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import numpy as np
torch.manual_seed(7)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
EPOCHS = 20
BATCH_SIZE = 64

# Load training dataset into a single batch to compute mean and stddev.
transform = transforms.Compose([transforms.ToTensor()])
trainset = MNIST(root='./pt_data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=len(trainset), shuffle=True)
data = next(iter(trainloader))
mean = data[0].mean()
stddev = data[0].std()

# Helper function needed to standardize data when loading datasets.
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(mean, stddev)])

trainset = MNIST(root='./pt_data', train=True, download=True, transform=transform)
testset = MNIST(root='./pt_data', train=False, download=True, transform=transform)


100%|██████████| 9912422/9912422 [00:00<00:00, 57380949.50it/s]


Extracting ./pt_data/MNIST/raw/train-images-idx3-ubyte.gz to ./pt_data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 58322433.23it/s]


Extracting ./pt_data/MNIST/raw/train-labels-idx1-ubyte.gz to ./pt_data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 20874003.66it/s]


Extracting ./pt_data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./pt_data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 10892240.58it/s]

Extracting ./pt_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./pt_data/MNIST/raw



In this example we want to use ReLU as activation for the first layer and softmax for the second layer. One thing that is slightly unintuitive is that we omit the activation function altogether for the output layer. The reason is that the cross-entropy loss function implementation in PyTorch is implemented using the inputs to the softmax (also known as logits) instead of the outputs. This makes for an implementation that is more numerically stable (also see some more details in the section "Computer implementation of the cross-entropy loss function" in Chapter 5 in the book).

Apart from changing the layer types, we also change the weight initialization scheme to Kaiming (He) for the ReLU layer and Xavier (Glorot) for the softmax layer.


In [3]:
# Define model. Final activation is omitted since softmax is part of
# cross-entropy loss function in PyTorch.
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(784, 25),
    nn.ReLU(),
    nn.Linear(25, 10)
)

# Retrieve layers for custom weight initialization.
layers = next(model.modules())
hidden_layer = layers[1]
output_layer = layers[3]

# Kaiming (He) initialization.
nn.init.kaiming_normal_(hidden_layer.weight)
nn.init.constant_(hidden_layer.bias, 0.0)

# Xavier (Glorot) initialization.
nn.init.xavier_uniform_(output_layer.weight)
nn.init.constant_(output_layer.bias, 0.0)


Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)

The training loop looks very similar to c5e1_mnist_learning. We change to using the Adam optimizer and the CrossEntropyLoss function.

Additionally, inside the training loop we no longer conver the targets to one-hot encoded targets. The reason is that the CrossEntropyLoss function object is implemented in a way where it assumes that the input is an integer specifying the index to be hot instead of assuming a one-hot encoded input.


In [4]:
# Use the Adam optimizer
# Cross-entropy loss as loss function.
optimizer = torch.optim.Adam(model.parameters())
loss_function = nn.CrossEntropyLoss()

# Transfer model to GPU
model.to(device)

# Create DataLoader objects that will help create mini-batches.
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
testloader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False)

# Train the model. In PyTorch we have to implement the training loop ourselves.
for i in range(EPOCHS):
    model.train() # Set model in training mode.
    train_loss = 0.0
    train_correct = 0
    train_batches = 0
    for inputs, targets in trainloader:
        # Move data to GPU.
        inputs, targets = inputs.to(device), targets.to(device)

        # Zero the parameter gradients.
        optimizer.zero_grad()

        # Forward pass.
        outputs = model(inputs)
        # Cross-entropy loss does not need one-hot targets in PyTorch.
        loss = loss_function(outputs, targets)

        # Accumulate metrics.
        _, indices = torch.max(outputs.data, 1)
        train_correct += (indices == targets).sum().item()
        train_batches +=  1
        train_loss += loss.item()

        # Backward pass and update.
        loss.backward()
        optimizer.step()

    train_loss = train_loss / train_batches
    train_acc = train_correct / (train_batches * BATCH_SIZE)

    # Evaluate the model on the test dataset. Identical to loop above but without
    # weight adjustment.
    model.eval() # Set model in inference mode.
    test_loss = 0.0
    test_correct = 0
    test_batches = 0
    for inputs, targets in testloader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        loss = loss_function(outputs, targets)
        _, indices = torch.max(outputs, 1)
        test_correct += (indices == targets).sum().item()
        test_batches +=  1
        test_loss += loss.item()

    test_loss = test_loss / test_batches
    test_acc = test_correct / (test_batches * BATCH_SIZE)

    print(f'Epoch {i+1}/{EPOCHS} loss: {train_loss:.4f} - acc: {train_acc:0.4f} - val_loss: {test_loss:.4f} - val_acc: {test_acc:0.4f}')


Epoch 1/20 loss: 0.3651 - acc: 0.8902 - val_loss: 0.2167 - val_acc: 0.9310
Epoch 2/20 loss: 0.1933 - acc: 0.9428 - val_loss: 0.1736 - val_acc: 0.9433
Epoch 3/20 loss: 0.1576 - acc: 0.9527 - val_loss: 0.1529 - val_acc: 0.9501
Epoch 4/20 loss: 0.1371 - acc: 0.9591 - val_loss: 0.1534 - val_acc: 0.9494
Epoch 5/20 loss: 0.1246 - acc: 0.9621 - val_loss: 0.1349 - val_acc: 0.9565
Epoch 6/20 loss: 0.1123 - acc: 0.9656 - val_loss: 0.1457 - val_acc: 0.9513
Epoch 7/20 loss: 0.1052 - acc: 0.9679 - val_loss: 0.1300 - val_acc: 0.9555
Epoch 8/20 loss: 0.0986 - acc: 0.9693 - val_loss: 0.1229 - val_acc: 0.9575
Epoch 9/20 loss: 0.0915 - acc: 0.9715 - val_loss: 0.1294 - val_acc: 0.9557
Epoch 10/20 loss: 0.0839 - acc: 0.9744 - val_loss: 0.1226 - val_acc: 0.9589
Epoch 11/20 loss: 0.0816 - acc: 0.9738 - val_loss: 0.1321 - val_acc: 0.9569
Epoch 12/20 loss: 0.0772 - acc: 0.9751 - val_loss: 0.1473 - val_acc: 0.9533
Epoch 13/20 loss: 0.0731 - acc: 0.9760 - val_loss: 0.1206 - val_acc: 0.9602
Epoch 14/20 loss: 0.0